# chap4 에지와 영역

#### 에지 
물체 경계에 위치한 점 
- 에지 검출 : 에지에 해당 화소 찾기
- 에지 향상 : 에지 더 잘보이도록 하기 위해 에지와 배경 간의 대비 증가
- 에지 추적 : 에지 따라가기

- 에지를 완벽하게 검출해 물체의 경계를 폐곡선으로 따낼 수 있다면 분할 문제 해결
- 특성이 크게 다른 화소에 집중하는 방식

-> 물체의 위치 모양 크기 에 대한 정보 찾기 가능

#### 영역
특성이 비슷한 화소를 묶는 방식

에지 검출 알고리즘 : 물체 내부는 명암이 서서히 변하고 경계는 급격히 변하는 특성 활용

### 4.1.1 영상의 미분
미분 : 변수의 x 값이 미세하게 증가했을 때 함수 변화량 측정, 에지 추출 방법

* 영상을 (x, y) 변수의 함수로 간주했을 때, 이 함수의 1차 미분(1st derivative) 값이 크게 나타나는 부분을 검출

디지털 영상 미분
- 미분은 기본적으로 연속적인 공간에서 적용, but 영상은 연속공간이 아닌 이산 공간이기에 근사화한 것으로 적용
- 영상 f에 미분 적용해 f'
- 필터 u(에지 연산자)로 컨볼루션하여 구현
- 명암을 미분해 튀어나온 부분이 에지

### 4.1.2 에지 연산자
- 명암 변화 x = 0
- 명암 변화 o = 3

컨볼루션의 값은 f 원래 영상의 부호를 의미한다 !?

물체 경계를 지나면서 명암값이 커지면 미분값 양수, 작아지면 음수


램프에지 : 계단 모양이 아닌, 명암이 몇화소에 걸쳐 변화, 정확한 에지의 위치 찾기 어려움




In [ ]:
# 4-1 에지 검출
import cv2 as cv

img=cv.imread('soccer.jpg')
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY) # 명암으로 변화

# 소벨 연산자 적용
grad_x=cv.Sobel(gray,cv.CV_32F,1,0,ksize=3)	# 결과 영상 32비트 실수 맵 저장, (1,0) x 방향 연산자, 3*3 크기 사용
grad_y=cv.Sobel(gray,cv.CV_32F,0,1,ksize=3) # y 방향 연산자

# 음수가 포함된 맵에 절대값을 취해 양수 영상으로 변환
sobel_x=cv.convertScaleAbs(grad_x)	# convertScaleAbs : 부호없는 8비트형 맵을 형성해 크기가 0보다 작으면 0, 255넘으면 255
sobel_y=cv.convertScaleAbs(grad_y)

# 에지 강도 계산
edge_strength=cv.addWeighted(sobel_x,0.5,sobel_y,0.5,0)	# sobel_x * 0.5 + sobel_y * 0 + 0
# sobel_x 와 sobel_y 가 같은 데이터 형이면 결과영상 같은 데이터 형
# 다른 데이터 형이면 결과영상 에러 

# 결과영상 윈도우 디스플레이
cv.imshow('Original',gray)
cv.imshow('sobelx',sobel_x)
cv.imshow('sobely',sobel_y)
cv.imshow('edge strength',edge_strength)

cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# 4-2 캐니 에지
# import cv2 as cv

img=cv.imread('soccer.jpg')	# 영상 읽기
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)


canny1=cv.Canny(gray,50,150)	# Tlow=50, Thigh=150으로 설정
canny2=cv.Canny(gray,100,200)	# Tlow=100, Thigh=200으로 설정


cv.imshow('Original',gray)
cv.imshow('Canny1',canny1) # 에지 강도가 작은 화소도 추적 가능하나 잡음 발생
cv.imshow('Canny2',canny2) # 임계값 높으면 에지강도가 큰 화소만 추적해 더 작은 에지 발생


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# 4-3 직선 검출
import cv2 as cv
import numpy as np


img=cv.imread('soccer.jpg')	 # 영상 읽기
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
canny=cv.Canny(gray,100,200) 


contour,hierarchy=cv.findContours(canny,cv.RETR_LIST,cv.CHAIN_APPROX_NONE) # 경계선 정보 검출


lcontour=[]   
for i in range(len(contour)):
    if contour[i].shape[0]>100:	# 길이가 100보다 크면 
        lcontour.append(contour[i])
  
# 외곽선 그리기
cv.drawContours(img,lcontour,-1,(0,255,0),3)
             

cv.imshow('Original with contours',img)    
cv.imshow('Canny',canny)    


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# 4-3 직선검출
import cv2 as cv 


img=cv.imread('apples.jpg')
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

apples=cv.HoughCircles(gray,cv.HOUGH_GRADIENT,1,200,param1=150,param2=20,minRadius=50,maxRadius=120) # 원 검출 허프변환

# 원의 중심, 반지름 정보 이용해 원래 영상에 원 그려넣기
for i in apples[0]: 
    cv.circle(img,(int(i[0]),int(i[1])),int(i[2]),(255,0,0),2)


cv.imshow('Apple detection',img)  


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
!pip install scikit-image

In [ ]:
# 4-4
import skimage # 입력 영상을 슈퍼화소로 분할하기에 slic 함수가 사용하기 편리
import numpy as np
import cv2 as cv


img = skimage.data.coffee() # 내부 coffee 영상 읽어 객체 저장
cv.imshow('Coffee image',cv.cvtColor(img,cv.COLOR_RGB2BGR))


slic1 = skimage.segmentation.slic(img,compactness=20,n_segments=600) # 슈퍼 화소 분할 수행
sp_img1 = skimage.segmentation.mark_boundaries(img,slic1) # 객체 분할 정보 img 영상에 표시, 결과를 sp_img1 객체에 저장
sp_img1 = np.uint8(sp_img1*255.0) # 표현 type을 0-255사이로 변환하고 unit8형으로 변환

# 위와 동일, 파라미터만 수정
slic2=skimage.segmentation.slic(img,compactness=40,n_segments=600) 
sp_img2=skimage.segmentation.mark_boundaries(img,slic2)
sp_img2=np.uint8(sp_img2*255.0)


cv.imshow('Super pixels (compact 20)',cv.cvtColor(sp_img1,cv.COLOR_RGB2BGR))
cv.imshow('Super pixels (compact 40)',cv.cvtColor(sp_img2,cv.COLOR_RGB2BGR))


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import skimage
import numpy as np
import cv2 as cv
import time


coffee=skimage.data.coffee()


start=time.time() # 분할하는 데 걸리는 시간 측정
slic=skimage.segmentation.slic(coffee,compactness=20,n_segments=600,start_label=1) # 슈퍼화소로 분할

g=skimage.future.graph.rag_mean_color(coffee,slic,mode='similarity') 
ncut=skimage.future.graph.cut_normalized(slic,g)	# 정규화 절단
print(coffee.shape,' Coffee 영상을 분할하는데 ',time.time()-start,'초 소요') # 시간 측정


marking=skimage.segmentation.mark_boundaries(coffee,ncut)
ncut_coffee=np.uint8(marking*255.0)


cv.imshow('Normalized cut',cv.cvtColor(ncut_coffee,cv.COLOR_RGB2BGR)) 


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv 
import numpy as np


img=cv.imread('soccer.jpg')	# 영상 읽기
img_show=np.copy(img)		# 붓 칠을 디스플레이할 목적의 영상

# 사용자가 붓칠에 따라 물체인지 배경인지에 대한 정보를 기록할 배열 생성
mask=np.zeros((img.shape[0],img.shape[1]),np.uint8) 
mask[:,:]=cv.GC_PR_BGD		# 모든 화소를 배경일 것 같음으로 초기화


BrushSiz=9				# 붓의 크기
LColor,RColor=(255,0,0),(0,0,255)	# 파란색(물체)과 빨간색(배경)


def painting(event,x,y,flags,param):
    if event==cv.EVENT_LBUTTONDOWN:   
        cv.circle(img_show,(x,y),BrushSiz,LColor,-1)	# 왼쪽 버튼 클릭하면 파란색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_FGD,-1)

    elif event==cv.EVENT_RBUTTONDOWN: 
        cv.circle(img_show,(x,y),BrushSiz,RColor,-1)	# 오른쪽 버튼 클릭하면 빨간색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_BGD,-1)

    elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_LBUTTON:
        cv.circle(img_show,(x,y),BrushSiz,LColor,-1)# 왼쪽 버튼 클릭하고 이동하면 파란색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_FGD,-1)

    elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_RBUTTON:
        cv.circle(img_show,(x,y),BrushSiz,RColor,-1)	# 오른쪽 버튼 클릭하고 이동하면 빨간색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_BGD,-1)

    cv.imshow('Painting',img_show)

    
cv.namedWindow('Painting')
cv.setMouseCallback('Painting',painting)


while(True):				# 붓 칠을 끝내려면 'q' 키를 누름
    if cv.waitKey(1)==ord('q'): 
        break

# ----------------- GrabCut 적용하는 코드 -----------------
background=np.zeros((1,65),np.float64)	# 배경 히스토그램 0으로 초기화
foreground=np.zeros((1,65),np.float64)	# 물체 히스토그램 0으로 초기화

# 실제 분할시도
cv.grabCut(img,mask,None,background,foreground,5,cv.GC_INIT_WITH_MASK)
mask2=np.where((mask==cv.GC_BGD)|(mask==cv.GC_PR_BGD),0,1).astype('uint8')
grab=img*mask2[:,:,np.newaxis]
cv.imshow('Grab cut image',grab)  


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import skimage
import numpy as np
import cv2 as cv


orig=skimage.data.horse()
img=255-np.uint8(orig)*255 # 말 영역은 255, 배경은 0
cv.imshow('Horse',img)

# 물체 경계선 추출
contours,hierarchy=cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)


img2=cv.cvtColor(img,cv.COLOR_GRAY2BGR)		# 컬러 디스플레이용 영상
cv.drawContours(img2,contours,-1,(255,0,255),2) # 경계선 표시 (영상,경계선,경계선 모두 표시, 색깔, 선 두께)
cv.imshow('Horse with contour',img2)


contour=contours[0]


m=cv.moments(contour)				# 모멘트 추출해 저장  
area=cv.contourArea(contour) # 경계선으로 둘러싸인 영역의 면적 계산
cx,cy=m['m10']/m['m00'],m['m01']/m['m00'] # 중점
perimeter=cv.arcLength(contour,True) # 둘레의 길이 계산
roundness=(4.0*np.pi*area)/(perimeter*perimeter) # 둥근정도
print('면적=',area,'\n중점=(',cx,',',cy,')','\n둘레=',perimeter,'\n둥근 정도=',roundness)



img3=cv.cvtColor(img,cv.COLOR_GRAY2BGR)		# 컬러 디스플레이용 영상



contour_approx=cv.approxPolyDP(contour,8,True)	# 직선 근사
cv.drawContours(img3,[contour_approx],-1,(0,255,0),2)


hull=cv.convexHull(contour)			# 볼록 헐
hull=hull.reshape(1,hull.shape[0],hull.shape[2])
cv.drawContours(img3,hull,-1,(0,0,255),2)


cv.imshow('Horse with line segments and convex hull',img3)


cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv
import numpy as np

img=np.array([[0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0],
              [0,0,0,1,1,0,0,0,0,0],
              [0,0,0,1,1,1,0,0,0,0],
              [0,0,0,1,1,1,1,0,0,0],
              [0,0,0,1,1,1,1,1,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0]],dtype=np.float32) # 입력영상 생성

# 미분 필터 생성
ux=np.array([[-1,0,1]])
uy=np.array([-1,0,1]).transpose()
# 가우시안 필터 생성
k=cv.getGaussianKernel(3,1)
g=np.outer(k,k.transpose())

# dy, dx 도출
dy=cv.filter2D(img,cv.CV_32F,uy)
dx=cv.filter2D(img,cv.CV_32F,ux)

dyy=dy*dy
dxx=dx*dx
dyx=dy*dx

gdyy=cv.filter2D(dyy,cv.CV_32F,g)
gdxx=cv.filter2D(dxx,cv.CV_32F,g)
gdyx=cv.filter2D(dyx,cv.CV_32F,g)

# 특징가능성 맵 도출
C=(gdyy*gdxx-gdyx*gdyx)-0.04*(gdyy+gdxx)*(gdyy+gdxx) 


# 비최대 억제 사용
for j in range(1,C.shape[0]-1):		
    for i in range(1,C.shape[1]-1):
        if C[j,i]>0.1 and sum(sum(C[j,i]>C[j-1:j+2,i-1:i+2]))==8: 
          # 극점이 되려면 C가 0.1보다 커야 하며 8개 이웃보다 커야 함
            img[j,i]=9			# 특징점을 원본 영상에 9로 표시
                
# 특징 가능성 맵 C 계산하는 데 필요한 미분 영상 출력                
np.set_printoptions(precision=2)
print(dy) 
print(dx) 
print(dyy) 
print(dxx) 
print(dyx) 
print(gdyy) 
print(gdxx) 
print(gdyx) 

print(C)					# 특징 가능성 맵 
print(img)					# 특징점을 9로 표시한 원본 영상 

# 화소 확인 가능하게 16배로 확대해 윈도우에 보임
popping=np.zeros([160,160],np.uint8)	
for j in range(0,160):
    for i in range(0,160):
        popping[j,i]=np.uint8((C[j//16,i//16]+0.06)*700)  

cv.imshow('Image Display2',popping)    
cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv

img=cv.imread('mot_color70.jpg') # 영상 읽기
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create() # SIFT 특징점 추출하는데 쓸 객체 생성
kp,des=sift.detectAndCompute(gray,None) # 특징점, 기술자 탐색

# 특징점 영상에 표시
gray=cv.drawKeypoints(gray,kp,None,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv.imshow('sift', gray)

k=cv.waitKey()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv
import numpy as np
import time

# 물체 모델 영상 정하기
img1=cv.imread('mot_color70.jpg')[190:350,440:560] # 버스를 크롭하여 모델 영상으로 사용
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)

# 물체 장면 영상 정하기
img2=cv.imread('mot_color83.jpg')			     
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)
print('특징점 개수:',len(kp1),len(kp2)) 

start=time.time()
flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED) # FLANN 라이브러리 사용
knn_match=flann_matcher.knnMatch(des1,des2,2) # 최근접 두개 찾기

# 임계값 이용해 최근접 이웃거리 비율 전략 적용
T=0.7
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)
print('매칭에 걸린 시간:',time.time()-start) 

# 매칭 결과 보여줄 ㄹ영상 생성
img_match=np.empty((max(img1.shape[0],img2.shape[0]),img1.shape[1]+img2.shape[1],3),dtype=np.uint8)
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('Good Matches', img_match)

k=cv.waitKey()
cv.destroyAllWindows()

In [ ]:

# good_match : 매칭 쌍 중 좋은 것을 골라 저장한 리스트
points1=np.float32([kp1[gm.queryIdx].pt for gm in good_match]) # 첫번째 영상 특징점 좌표 
points2=np.float32([kp2[gm.trainIdx].pt for gm in good_match]) # 두번째 영상 특징점 좌표

H,_=cv.findHomography(points1,points2,cv.RANSAC) # 호모그래피 행렬 추정

h1,w1=img1.shape[0],img1.shape[1] 		# 첫 번째 영상의 크기
h2,w2=img2.shape[0],img2.shape[1] 		# 두 번째 영상의 크기

box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2) # 첫번째 영상을 포함하는 네 구석의 좌표 저장
box2=cv.perspectiveTransform(box1,H) # 첫번째 영상 좌표에 행렬 H 적용, 두번째 영상 투영한 결과 저장

img2=cv.polylines(img2,[np.int32(box2)],True,(0,255,0),8) # box2를 두번째 영상에 그리기

img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8)
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
   
cv.imshow('Matches and Homography',img_match)

k=cv.waitKey()
cv.destroyAllWindows()